In [1]:
import os
print(os.getcwd())
os.chdir("../")
print(os.getcwd())

/home/ryefoxlime/MathTutor/research
/home/ryefoxlime/MathTutor


Update Entity File

In [2]:
from dataclasses import dataclass

@dataclass
class DataStoring:
    dimension: int
    collection_name: str
    contexts: list[str]
    sources: set

Update Config Manager

In [ ]:
from src.Math.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.Math.utils.common import read_yaml

class ConfigurationManager:
    def __init__(
        self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

    def get_data_storing_params(self) -> DataStoring:
        params = self.params.data_storing

        data_storing_params = DataStoring(
            dimension=params.dimension,
            collection_name=params.collection_name,
            contexts = params.contexts,
            sources = params.sources
        )
        return data_storing_params

Update Components

In [5]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct


class QdrantStorage:
    def __init__(self, config: DataStoring, url = "http://localhost:6333"):
        self.config = config
        self.client = QdrantClient(url=url, timeout=30)
        
        if not self.client.collection_exists(self.config.collection_name):
            self.client.create_collection(
                collection_name=self.config.collection_name,
                vectors_config=VectorParams(size=self.config.dimension, distance=Distance.COSINE),
            )

    def upsert(self, ids, vectors, payloads):
        points = [
            PointStruct(id=ids[i], vector=vectors[i], payload=payloads[i])
            for i in range(len(ids))
        ]
        self.client.upsert(self.config.collection_name, points=points)

    def search(self, query_vector, top_k: int = 5):
        results = self.client.search(
            collection_name=self.config.collection_name,
            query_vector=query_vector,
            with_payload=True,
            limit=top_k,
        )

        for r in results:
            payload = getattr(r, "payload", None) or {}
            text = payload.get("text", "")
            source = payload.get("source", "")
            if text:
                self.config.contexts.append(text)
                self.config.sources.add(source)

        return {"contexts": self.config.contexts, "sources": list(self.config.sources)}
